In [1]:
pip install keras-tuner

     |████████████████████████████████| 71kB 2.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=8bb882862a6054ac1e02f09e0798a7b91dfe51374eab15e74cd27760a7adcdaf
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=105214e4650cd44379f167665584835f2dfcf63907c1419098e1d1ab7bb82b2f
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model=keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')#output layer
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 09m 18s]
val_accuracy: 0.10499999672174454

Best val_accuracy So Far: 0.8321666717529297
Total elapsed time: 01h 08m 54s

Search: Running Trial #5

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |112               |80                
conv_1_kernel     |3                 |3                 
conv_2_filter     |48                |48                
conv_2_kernel     |5                 |3                 
dense_1_units     |64                |48                
learning_rate     |0.001             |0.001             

Epoch 1/3
1688/1688 [==============================] - 632s 374ms/step - loss: 0.7712 - accuracy: 0.7113 - val_loss: 0.5656 - val_accuracy: 0.7873
Epoch 2/3
   5/1688 [..............................] - ETA: 10:01 - loss: 0.6292 - accuracy: 0.7875

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-16-14d18f8bc01c>", line 1, in <module>
    tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/base_tuner.py", line 131, in search
    self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/multi_execution_tuner.py", line 96, in run_trial
    history = self._build_and_fit_model(trial, fit_args, copied_fit_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/kerastuner/engine/tuner.py", line 141, in _build_and_fit_model
    return model.fit(*fit_args, **fit_kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py", line 1183, in fit
    tmp_logs = self.train_function(iterator)
  File "/u

KeyboardInterrupt: ignored

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        34608     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,363,050
Trainable params: 1,363,050
Non-trainable params: 0
_________________________________________________________________
